## OpenFermion FermionOperator Integration

You can use Docker to easily run this notebook: 

```bash
$ docker run --name xacc-notebook -it -p 8888:8888 -d -v $(pwd):/home/notebooks mccaskey/xacc-all-gate-jupyter-fc26
$ (to see jupyter web address) docker logs xacc-notebook
```

Copy and paste the Jupyter URL into your web browser. You now have a fully configure XACC instance with XACC-IBM, XACC-Rigetti, XACC-Python, TNQVM, and XACC-VQE plugins. 

In [1]:
from openfermion.ops import FermionOperator
import sys
sys.path.append('/usr/local/xacc/lib/python')
import pyxaccvqe as vqe

op = FermionOperator('', 0.713776) + \
   FermionOperator('0^ 0', -1.25248) + \
   FermionOperator('0^ 1^ 1 0', 0.337247) + \
   FermionOperator('0^ 1^ 3 2', 0.0906438) + \
   FermionOperator('0^ 2^ 0 2', 0.0906438) + \
   FermionOperator('0^ 2^ 2 0', 0.331736) + \
   FermionOperator('0^ 3^ 1 2', 0.0906438) + \
   FermionOperator('0^ 3^ 3 0', 0.331736) + \
   FermionOperator('1^ 0^ 0 1', 0.337247) + \
   FermionOperator('1^ 0^ 2 3', 0.0906438) + \
   FermionOperator('1^ 1', -1.25248) + \
   FermionOperator('1^ 2^ 0 3', 0.0906438) + \
   FermionOperator('1^ 2^ 2 1', 0.331736) + \
   FermionOperator('1^ 3^ 1 3', 0.0906438) + \
   FermionOperator('1^ 3^ 3 1', 0.331736) + \
   FermionOperator('2^ 0^ 0 2', 0.331736) + \
   FermionOperator('2^ 0^ 2 0', 0.0906438) + \
   FermionOperator('2^ 1^ 1 2', 0.331736) + \
   FermionOperator('2^ 1^ 3 0', 0.0906438) + \
   FermionOperator('2^ 2', -0.475934) + \
   FermionOperator('2^ 3^ 1 0', 0.0906438) + \
   FermionOperator('2^ 3^ 3 2', 0.348699) + \
   FermionOperator('3^ 0^ 0 3', 0.331736) + \
   FermionOperator('3^ 0^ 2 1', 0.0906438) + \
   FermionOperator('3^ 1^ 1 3', 0.331736) + \
   FermionOperator('3^ 1^ 3 1', 0.0906438) + \
   FermionOperator('3^ 2^ 0 1', 0.0906438) + \
   FermionOperator('3^ 2^ 2 3', 0.348699) + \
   FermionOperator('3^ 3', -0.475934)

# Convert op to a Pauli hamiltonian, and diagonalize 
# that hamiltonian with Eigen
print('Diagonalized E = ', vqe.execute(op).results[0][1])

print('UCCSD VQE E = ', vqe.execute(op, **{'task':'vqe', 'n-electrons':2}).results[0][1])

print('Jordan-Wigner with XACC:\n', vqe.compile(op))


/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Diagonalized E =  -1.137274736593949
UCCSD VQE E =  -1.1372726844865666
Jordan-Wigner with XACC:
 (0.174349,0) Z2 Z3 + (0.120546,0) Z1 Z3 + (-0.222797,0) Z3 + (0.165868,0) Z0 Z3 + (0.120546,0) Z0 Z2 + (0.0453219,0) X0 Y1 Y2 X3 + (-0.0453219,0) Y0 Y1 X2 X3 + (0.0453219,0) Y0 X1 X2 Y3 + (0.165868,0) Z1 Z2 + (-0.0453219,0) X0 X1 Y2 Y3 + (0.171202,0) Z1 + (0.168624,0) Z0 Z1 + (-0.222797,0) Z2 + (0.171202,0) Z0 + (-0.0988368,0)
